#  The SWAG Algorithm; a Mathematical Approach that Outperforms Traditional Deep Learning. Theory and Implementation



##           solving any classification problems with just one layer neural network

### **Saeid Safaei, PhD Candidate**
#### 12/17/2018

### 1. Introduction to SWAG**


The link of Paper

https://arxiv.org/pdf/1811.11813.pdf

####  Suppose we have input P and Lable Y.
####  There are some functions like F(x), G(X), H(x), T(X) which map input  P to output Y. The question is how to find those functions?

<img src="1.png" width="40%" height="40%"></img>

####  Moreover, we can approximate a function with polynomials
####  So, if we find a_{ij} then the problem solved. How find those coefficients?


<img src="2.png" width="45%" height="45%"></img>

####  We push the inputs to the formula. Now we have the following.


<img src="3.png" width="60%" height="60%"></img>

####  If we solve the following problem, we can find those coefficients. P and Y are constant. 

<img src="4.png"  width="50%" height="50%" ></img>

### Traditional deep learning

<img src="5.png" ></img>

#### Our model with 1 Layer

<img src="6.png" ></img>

#### Our model with 2 Layers

<img src="7.png" ></img>

### Import Libraries

In [1]:
import timeit
import numpy as np
import pandas as pd
from keras.models import Sequential
from sklearn.model_selection import KFold,train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from pandas import read_csv
from keras.datasets import mnist
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.initializers import RandomNormal,glorot_normal
from sklearn.model_selection import StratifiedKFold
np.random.seed(7)


C:\Users\BRG_user\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### To Run the code you need to download this file. it is the implementation of SWAG Layer.

In [2]:
from Dense_SWAG import Dense_Co


### Define normalize Functions

### Define Rescaling Functions from 0-1 to 0.1-0.9


In [3]:
#Define normalize Functions
def normalize(d):
    # d is a (n x dimension) np array
    d -= np.min(d, axis=0)
    d /= np.ptp(d, axis=0)
    return d

#Define Rescaling Functions from 0-1 to 0.1-0.9
def rescale_range(d):
    # d is a (n x dimension) np array
    d=np.multiply(d, 0.89)
    d=np.add(d, 0.01)
    return d

### This method compiles a Keras model and return it.
### input_dim is the dimension of input and  output_dim is the dimension of output.
### In our model image the number of Green, Red, and Blue are equal, and we show them by hidden_dim
### by default hidden_dim = input_dim but we can change it.

In [4]:
def build_model_v1(input_dim,hidden_dim,output_dim):
    model = Sequential()
    model.add(Dense_Co(output_dim,hidden_dim=hidden_dim ,input_dim=input_dim))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    return model


def build_model(input_dim,hidden_dim,output_dim):
    model = Sequential()
    model.add(Dense_Co(output_dim,hidden_dim=hidden_dim ,input_dim=input_dim,  kernel_initializer=RandomNormal(
            mean=0.0, stddev=0.04), bias_initializer=RandomNormal(mean=0.0, stddev=0.04)))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    return model

In [5]:
epochs=5
batch_size=1

# define 10-fold cross validation test harness
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
cvscores = []

#  ========================================================
# First experiment
##  Load data of   ionosphere data set


This is a small dataset that you can <a href="https://www.kaggle.com/creepyghost/uci-ionosphere/version/1">download from the kaggle </a>.


### About this file:

Source: Donated to UCI Machine Learning Repository by

Donor:

Vince Sigillito (vgs '@' aplcen.apl.jhu.edu)

Source:

Space Physics Group Applied Physics Laboratory Johns Hopkins University Johns Hopkins Road Laurel, MD 20723

In [6]:
dataframe = read_csv("data\ionosphere_data_kaggle.csv", header=1)
dataset = dataframe.values

#Number of Rows and columns
Number_rows,Input_size=dataset.shape

X = dataset[:,0:Input_size-1].astype(float)
Y = dataset[:,Input_size-1]
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
Y = encoder.transform(Y)


### We use 10 k-fold Cross-Validation

In [7]:
cvscores = []

# create model
model=build_model(Input_size-1,Input_size-1,1)
model.summary()

for train, test in kfold.split(X, Y):
    model.fit(X[train], Y[train], epochs=epochs, batch_size=batch_size, verbose=0)
    scores = model.evaluate(X[test], Y[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("The Mean accuracy is %.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense__co_1 (Dense_Co)       (None, 1)                 7345      
Total params: 7,345
Trainable params: 7,345
Non-trainable params: 0
_________________________________________________________________
acc: 88.89%
acc: 97.22%
acc: 97.22%
acc: 97.22%
acc: 97.14%
acc: 97.14%
acc: 97.06%
acc: 97.06%
acc: 97.06%
acc: 97.06%
The Mean accuracy is 96.31% (+/- 2.47%)


#  ========================================================
# Second experiment
##  Load data of   Iris Data Set


This is a small dataset that you can <a href="https://www.kaggle.com/uciml/iris#Iris.csv">download from the kaggle </a>.


### About this file:

https://www.kaggle.com/uciml/iris#Iris.csv


In [8]:
dataframe = read_csv("data\iris.csv", header=1)
dataset = dataframe.values


#Number of Rows and columns
Number_rows,Input_size=dataset.shape

X = dataset[:,0:Input_size-1].astype(float)
Y = dataset[:,Input_size-1]
X=normalize(X)
X=rescale_range(X)

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
Y = encoder.transform(Y)


In [9]:
cvscores = []

# create model
model=build_model(Input_size-1,Input_size-1,1)
model.summary()

for train, test in kfold.split(X, Y):
    model.fit(X[train], Y[train], epochs=epochs, batch_size=batch_size, verbose=0)
    scores = model.evaluate(X[test], Y[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("The Mean accuracy is %.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense__co_2 (Dense_Co)       (None, 1)                 211       
Total params: 211
Trainable params: 211
Non-trainable params: 0
_________________________________________________________________
acc: 93.33%
acc: 93.33%
acc: 93.33%
acc: 100.00%
acc: 100.00%
acc: 93.33%
acc: 100.00%
acc: 100.00%
acc: 100.00%
acc: 100.00%
The Mean accuracy is 97.33% (+/- 3.27%)


#  ========================================================
# Third experiment
##  Load data of  banknote authentication Data Set 

This is a small dataset that you can <a href="https://www.kaggle.com/jacksonharper/data_banknote_authentication">download from the kaggle </a>.


### About this file:

https://www.kaggle.com/jacksonharper/data_banknote_authentication


In [10]:
dataframe = read_csv("data\data_banknote_authentication.csv", header=None)
dataset = dataframe.values


#Number of Rows and columns
Number_rows,Input_size=dataset.shape

X = dataset[:,0:Input_size-1].astype(float)
Y = dataset[:,Input_size-1]
X=normalize(X)
X=rescale_range(X)

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
Y = encoder.transform(Y)


In [11]:
cvscores = []

# create model
model=build_model(Input_size-1,Input_size-1,1)
model.summary()

for train, test in kfold.split(X, Y):
    model.fit(X[train], Y[train], epochs=epochs, batch_size=batch_size, verbose=0)
    scores = model.evaluate(X[test], Y[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("The Mean accuracy is %.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense__co_3 (Dense_Co)       (None, 1)                 145       
Total params: 145
Trainable params: 145
Non-trainable params: 0
_________________________________________________________________
acc: 96.38%
acc: 94.93%
acc: 98.54%
acc: 97.81%
acc: 95.62%
acc: 97.81%
acc: 98.54%
acc: 99.27%
acc: 94.16%
acc: 98.54%
The Mean accuracy is 97.16% (+/- 1.67%)


#  ========================================================
# Fourth experiment
##  Load data of  Connectionist Bench (Sonar)

This is a small dataset that you can <a href="https://www.kaggle.com/adx891/sonar-data-set">download from the kaggle </a>.


### About this file:

Connectionist Bench (Sonar, Mines vs. Rocks) Data Set Download: Data Folder, Data Set Description


In [12]:
dataframe = read_csv("data\sonar.all-data.csv", header=None)
dataset = dataframe.values


#Number of Rows and columns
Number_rows,Input_size=dataset.shape

X = dataset[:,0:Input_size-1].astype(float)
Y = dataset[:,Input_size-1]
X=normalize(X)
X=rescale_range(X)

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
Y = encoder.transform(Y)


In [13]:
cvscores = []

# create model
model=build_model(Input_size-1,Input_size-1,1)
model.summary()

for train, test in kfold.split(X, Y):
    model.fit(X[train], Y[train], epochs=epochs, batch_size=batch_size, verbose=0)
    scores = model.evaluate(X[test], Y[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("The Mean accuracy is %.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense__co_4 (Dense_Co)       (None, 1)                 22321     
Total params: 22,321
Trainable params: 22,321
Non-trainable params: 0
_________________________________________________________________
acc: 81.82%
acc: 85.71%
acc: 90.48%
acc: 95.24%
acc: 100.00%
acc: 95.24%
acc: 100.00%
acc: 100.00%
acc: 100.00%
acc: 100.00%
The Mean accuracy is 94.85% (+/- 6.37%)


#  ========================================================
# Fifth  experiment    
##  Load data of  Pima Indians Diabetes Database



This is a small dataset that you can <a href="https://www.kaggle.com/uciml/pima-indians-diabetes-database">download from the kaggle </a>.


### About this file:

The datasets consist of several medical predictor (independent) variables and one target (dependent) variable, Outcome. Independent variables include the number of pregnancies the patient has had, their BMI, insulin level, age, and so on.

In [14]:
dataframe = read_csv("data\diabetes.csv", header=1)
dataset = dataframe.values


#Number of Rows and columns
Number_rows,Input_size=dataset.shape

X = dataset[:,0:Input_size-1].astype(float)
Y = dataset[:,Input_size-1]
X=normalize(X)
X=rescale_range(X)

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
Y = encoder.transform(Y)


In [15]:
cvscores = []

# create model
model=build_model(Input_size-1,Input_size-1,1)
model.summary()

for train, test in kfold.split(X, Y):
    model.fit(X[train], Y[train], epochs=epochs, batch_size=batch_size, verbose=0)
    scores = model.evaluate(X[test], Y[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("The Mean accuracy is %.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense__co_5 (Dense_Co)       (None, 1)                 481       
Total params: 481
Trainable params: 481
Non-trainable params: 0
_________________________________________________________________
acc: 68.83%
acc: 77.92%
acc: 80.52%
acc: 83.12%
acc: 83.12%
acc: 79.22%
acc: 75.32%
acc: 73.68%
acc: 75.00%
acc: 78.95%
The Mean accuracy is 77.57% (+/- 4.22%)


#  ========================================================
# Sixth  experiment        
##  Load data of  Red Wine Quality


This is a small dataset that you can download 
<a href="https://www.kaggle.com/uciml/red-wine-quality-cortez-et-al-2009#winequality-red.csv">
Here </a>.


### About this file:

This datasets is related to red variants of the Portuguese "Vinho Verde" wine. For more details, consult the reference [Cortez et al., 2009]. Due to privacy and logistic issues, only physicochemical (inputs) and sensory (the output) variables are available (e.g. there is no data about grape types, wine brand, wine selling price, etc.).

In [16]:
dataframe = read_csv("data\winequality-red.csv", header=1)
dataset = dataframe.values

#Number of Rows and columns
Number_rows,Input_size=dataset.shape

X = dataset[:,0:Input_size-1].astype(float)
Y = dataset[:,Input_size-1]
X=normalize(X)
X=rescale_range(X)

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
Y = encoder.transform(Y)


In [17]:
cvscores = []

# create model
model=build_model(Input_size-1,Input_size-1,1)
model.summary()

for train, test in kfold.split(X, Y):
    model.fit(X[train], Y[train], epochs=epochs, batch_size=batch_size, verbose=0)
    scores = model.evaluate(X[test], Y[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("The Mean accuracy is %.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense__co_6 (Dense_Co)       (None, 1)                 859       
Total params: 859
Trainable params: 859
Non-trainable params: 0
_________________________________________________________________
acc: 58.39%
acc: 54.66%
acc: 59.63%
acc: 60.62%
acc: 53.12%
acc: 63.12%
acc: 58.75%
acc: 60.62%
acc: 51.90%
acc: 59.87%
The Mean accuracy is 58.07% (+/- 3.45%)


### ========================================================
## Seventh  experiment
Load data of THE MNIST DATABASE of handwritten digits

### About this file:

    
The MNIST database of handwritten digits, has a training set of 60,000 examples, and a test set of 10,000 examples. It is a subset of a larger set available from NIST. The digits have been size-normalized and centered in a fixed-size image.


In [18]:
batch_size = 128
num_classes = 10
epochs= 4

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train+10
x_test = x_test+10

x_train /= 300
x_test /= 300
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

number_train=60000 
number_test=10000

x_train=x_train[0:number_train,:]
x_test=x_test[0:number_test,:]

y_train=y_train[0:number_train]
y_test=y_test[0:number_test]

60000 train samples
10000 test samples


In [19]:
models=build_model(784,500,10)
models.summary()
models.fit(x_train, y_train,validation_data=(x_test, y_test), epochs=epochs, batch_size=batch_size)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense__co_7 (Dense_Co)       (None, 10)                2385010   
Total params: 2,385,010
Trainable params: 2,385,010
Non-trainable params: 0
_________________________________________________________________
Train on 60000 samples, validate on 10000 samples
Epoch 1/4
60000/60000 [==============================] - 15s 244us/step - loss: 0.0205 - acc: 0.9431 - val_loss: 0.0134 - val_acc: 0.9696
Epoch 2/4
60000/60000 [==============================] - 14s 227us/step - loss: 0.0117 - acc: 0.9772 - val_loss: 0.0113 - val_acc: 0.9751
Epoch 3/4
60000/60000 [==============================] - 14s 227us/step - loss: 0.0097 - acc: 0.9832 - val_loss: 0.0106 - val_acc: 0.9766
Epoch 4/4
60000/60000 [==============================] - 14s 228us/step - loss: 0.0084 - acc: 0.9875 - val_loss: 0.0097 - val_acc: 0.9787
